In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.distributions import Normal
import torch.nn.functional as F
from matplotlib import pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Симулятор

Импорт моделей энергосети.

In [ ]:
from power_models import PowerGrid

Импорт моделей агентов.

Импорт функций.

In [ ]:
from general_func import reward_func

## Диапазоны параметров

In [3]:
seed = 1991
rng = np.random.default_rng(seed)

# Bus.
Ubus_nom = 48.0
Ubus_init = 0.95*Ubus_nom
beta_bus = 0.01e-3/(6*6*1800e-6) # dt/C

# Sources.
N_SOURCE = 5
Usource_init_min, Usource_init_max = [0.10*Ubus_nom, 0.95*Ubus_nom]
Rdroop_min, Rdroop_max = [2e-3, 20e-3]
beta_source_min, beta_source_max = [0.8, 0.9]
Uref_rnd_min, Uref_rnd_max = [0.95*Ubus_nom, 1.05*Ubus_nom]

Usource_init_set = rng.uniform(Usource_init_min, Usource_init_max, size=N_SOURCE)
Rdroop_set = rng.uniform(Rdroop_min, Rdroop_max, size=N_SOURCE)
beta_source_set = rng.uniform(beta_source_min, beta_source_max, size=N_SOURCE)
Uref_set = rng.uniform(Uref_rnd_min, Uref_rnd_max, size=N_SOURCE)

# Loads.
N_LOAD = 10
Prnd_min, Prnd_max = [10.0, 8.5e3]
beta_load_min, beta_load_max = [0.1, 0.9]

beta_load_set = rng.uniform(beta_load_min, beta_load_max, size=N_LOAD)
Uload_nom_set = np.full_like(beta_load_set, Ubus_nom)
Iload_init_set = np.full_like(beta_load_set, 0.0)


Конфигурация энергосети.

In [4]:
microgrid = PowerGrid(
    N_SOURCE = N_SOURCE,
    N_LOAD = N_LOAD,
    Rdroop_set = Rdroop_set,
    Ubus_init = Ubus_init,
    Usource_init_set = Usource_init_set,
    Uload_nom_set = Uload_nom_set,
    Iload_init_set =Iload_init_set,
    beta_bus = beta_bus,
    beta_source_set = beta_source_set,
    beta_load_set = beta_load_set
)

In [5]:
N_STEPS = 200
N_TRAIN = 10
seed_generator = rng.integers(0, 2**32, size=N_TRAIN, dtype=np.uint32)

Основной цикл симуляции.

In [6]:
bus_voltage = []
for m in range(N_TRAIN):
    load_random = np.random.default_rng(seed_generator[m])
    
    Pload_set = load_random.uniform(Prnd_min, Prnd_max, size=N_LOAD)
    beta_load_set = load_random.uniform(beta_load_min, beta_load_max, size=N_LOAD)
    trans_vals, steady_vals, agent_states, global_state = microgrid.step(
        N_STEPS = N_STEPS,
        Uref_set = Uref_set,
        Pload_set = Pload_set)
    
    bus_voltage.extend(trans_vals[0])
    
    print(f'Мощность системы электроснабжения: {np.sum(Pload_set):.2f}')
    print(f'Напряжение на шине, В:, {steady_vals[0]:.2f}')
    print(f'Токи источников, А:', np.round(steady_vals[1],2))
    print(f'Напряжения источников, В:', np.round(steady_vals[2],2))
    for idx, agent in enumerate(agent_states):
        print(f'Состояния агента {idx}:', np.round(agent, 2))
    print(f'Состояние микрогрида:', np.round(global_state,2))


Мощность системы электроснабжения: 51328.25
Напряжение на шине, В:, 45.70
Токи источников, А: [300.75 391.23 129.33   7.22 189.52]
Напряжения источников, В: [49.5  49.28 46.88 45.8  48.75]
Состояния агента 0: [300.75 189.52 391.23  49.5   48.75  49.28]
Состояния агента 1: [391.23 300.75 129.33  49.28  49.5   46.88]
Состояния агента 2: [129.33 391.23   7.22  46.88  49.28  45.8 ]
Состояния агента 3: [  7.22 129.33 189.52  45.8   46.88  48.75]
Состояния агента 4: [189.52   7.22 300.75  48.75  45.8   49.5 ]
Состояние микрогрида: [300.75 391.23 129.33   7.22 189.52  49.5   49.28  46.88  45.8   48.75]
Мощность системы электроснабжения: 43114.89
Напряжение на шине, В:, 46.06
Токи источников, А: [272.   351.58  89.57 -18.19 166.99]
Напряжения источников, В: [49.5  49.28 46.88 45.8  48.75]
Состояния агента 0: [272.   166.99 351.58  49.5   48.75  49.28]
Состояния агента 1: [351.58 272.    89.57  49.28  49.5   46.88]
Состояния агента 2: [ 89.57 351.58 -18.19  46.88  49.28  45.8 ]
Состояния агента

In [ ]:
# ===========================
# 2. Safe Action Mechanism (DEAA)
# ===========================
class SafeActionMechanism:
    def __init__(self, eta=0.5, phi=0.2, U_ref=50.0):
        self.eta = eta
        self.phi = phi
        self.L = U_ref
        self.T = 0.0

    def smooth(self, raw_action):
        L_new = self.eta * raw_action + (1 - self.eta) * (self.L + self.T)
        T_new = self.phi * (L_new - self.L) + (1 - self.phi) * self.T
        safe = L_new + T_new
        self.L, self.T = L_new, T_new
        return safe

# ===========================
# 3. Reward Function (Algorithm 1)
# ===========================


# ===========================
# 4. State Normalizer
# ===========================
class RunningNormalizer:
    def __init__(self, shape):
        self.mean = np.zeros(shape, dtype=np.float32)
        self.var = np.ones(shape, dtype=np.float32)
        self.count = 1e-4

    def update(self, x):
        batch_mean = np.mean(x, axis=0)
        batch_var = np.var(x, axis=0)
        batch_count = x.shape[0]

        delta = batch_mean - self.mean
        tot_count = self.count + batch_count
        self.mean += delta * batch_count / tot_count
        m_a = self.var * self.count
        m_b = batch_var * batch_count
        M2 = m_a + m_b + np.square(delta) * self.count * batch_count / tot_count
        self.var = M2 / tot_count
        self.count = tot_count

    def normalize(self, x):
        return (x - self.mean) / (np.sqrt(self.var) + 1e-8)

# ===========================
# 5. SMAPPO Agent
# ===========================
class SMAPPO:
    def __init__(self, obs_dim, action_dim, global_dim, lr_actor=1.5e-4, lr_critic=1e-4,
                 clip_eps=0.2, ent_coef=0.01, gamma=0.8, gae_lambda=0.95):
        self.actor = Actor(obs_dim, action_dim).to(device)
        self.critic = Critic(global_dim).to(device)
        self.actor_opt = optim.Adam(self.actor.parameters(), lr=lr_actor)
        self.critic_opt = optim.Adam(self.critic.parameters(), lr=lr_critic)
        self.clip_eps = clip_eps
        self.ent_coef = ent_coef
        self.gamma = gamma
        self.gae_lambda = gae_lambda

    def compute_gae(self, rewards, values, dones, next_value):
        advantages = []
        gae = 0
        for i in reversed(range(len(rewards))):
            if i == len(rewards) - 1:
                next_val = next_value
            else:
                next_val = values[i + 1]
            delta = rewards[i] + self.gamma * next_val * (1 - dones[i]) - values[i]
            gae = delta + self.gamma * self.gae_lambda * (1 - dones[i]) * gae
            advantages.insert(0, gae)
        advantages = torch.tensor(advantages, dtype=torch.float32, device=device)
        returns = advantages + torch.tensor(values, dtype=torch.float32, device=device)
        return advantages, returns

    def update(self, data, epochs=10, batch_size=64):
        obs = torch.tensor(data['obs'], dtype=torch.float32, device=device)
        global_obs = torch.tensor(data['global_obs'], dtype=torch.float32, device=device)
        actions = torch.tensor(data['actions'], dtype=torch.float32, device=device)
        logprobs = torch.tensor(data['logprobs'], dtype=torch.float32, device=device)
        returns = data['returns']
        advantages = data['advantages']

        n = len(obs)
        indices = np.arange(n)

        for _ in range(epochs):
            np.random.shuffle(indices)
            for start in range(0, n, batch_size):
                idx = indices[start:start+batch_size]
                obs_b = obs[idx]
                global_obs_b = global_obs[idx]
                actions_b = actions[idx]
                logprobs_b = logprobs[idx]
                returns_b = returns[idx]
                advantages_b = advantages[idx]

                # Normalize advantages
                advantages_b = (advantages_b - advantages_b.mean()) / (advantages_b.std() + 1e-8)

                # Critic loss
                values_pred = self.critic(global_obs_b)
                critic_loss = F.mse_loss(values_pred, returns_b)

                self.critic_opt.zero_grad()
                critic_loss.backward()
                self.critic_opt.step()

                # Actor loss
                mean_new, log_std_new = self.actor(obs_b)
                std_new = log_std_new.exp()
                dist_new = Normal(mean_new, std_new)
                logp_new = dist_new.log_prob(actions_b).sum(-1, keepdim=True)
                logp_new -= (2 * (np.log(2) - torch.tanh(actions_b) - F.softplus(-2 * torch.tanh(actions_b)))).sum(-1, keepdim=True)

                ratio = torch.exp(logp_new - logprobs_b)
                surr1 = ratio * advantages_b
                surr2 = torch.clamp(ratio, 1 - self.clip_eps, 1 + self.clip_eps) * advantages_b
                actor_loss = -torch.min(surr1, surr2).mean()

                entropy = dist_new.entropy().sum(-1).mean()
                actor_loss -= self.ent_coef * entropy

                self.actor_opt.zero_grad()
                actor_loss.backward()
                self.actor_opt.step()

# ===========================
# 6. Имитация среды (замените на реальную)
# ===========================
def mock_env_step(safe_actions, n_agents=4):
    # Моделируем изменения напряжения и тока
    base_U = 50.0
    base_I = np.random.uniform(-10, 10, n_agents)
    U = base_U + np.random.normal(0, 0.5, n_agents) + 0.01 * np.sum(safe_actions - base_U)
    I = base_I + np.random.normal(0, 0.2, n_agents)

    # Наблюдения: [I_self, I_neigh1, I_neigh2, U_self, U_neigh1, U_neigh2]
    obs_list = []
    for i in range(n_agents):
        I_self = I[i]
        U_self = U[i]
        I_n = [I[(i-1)%n_agents], I[(i+1)%n_agents]]
        U_n = [U[(i-1)%n_agents], U[(i+1)%n_agents]]
        obs = np.array([I_self] + I_n + [U_self] + U_n, dtype=np.float32)
        obs_list.append(obs)

    global_obs = np.concatenate(obs_list, dtype=np.float32)
    local_obs = np.stack(obs_list)
    done = False
    return local_obs, global_obs, U, I, done

# ===========================
# 7. Основной цикл обучения
# ===========================
if __name__ == "__main__":
    n_agents = 4
    obs_dim = 6
    action_dim = 1
    global_dim = n_agents * obs_dim
    max_steps = 400
    tau = 20
    batch_size = 168

    agent = SMAPPO(obs_dim, action_dim, global_dim, batch_size=batch_size)
    safe_mechs = [SafeActionMechanism() for _ in range(n_agents)]
    state_norm = RunningNormalizer(obs_dim)

    num_episodes = 1000
    for ep in range(num_episodes):
        # Reset
        local_obs, global_obs, _, _, _ = mock_env_step(np.full(n_agents, 50.0))
        count = 0
        buffer = {k: [] for k in ['obs', 'global_obs', 'actions', 'logprobs', 'rewards', 'dones', 'values']}

        for t in range(max_steps):
            # Нормализуем входы
            local_obs_norm = np.array([state_norm.normalize(o) for o in local_obs])
            actions_raw = []
            logprobs_raw = []
            safe_actions = []

            for i in range(n_agents):
                o = torch.tensor(local_obs_norm[i:i+1], device=device)
                a, logp = agent.actor.get_action(o)
                a_cpu = a.cpu().item()
                a_v = a_cpu * 10 + 50  # [-1,1] → [40,60]
                a_safe = safe_mechs[i].smooth(a_v)
                actions_raw.append(a_cpu)
                logprobs_raw.append(logp.item())
                safe_actions.append(a_safe)

            # Шаг среды
            next_local, next_global, voltages, currents, done = mock_env_step(safe_actions)
            reward, count = compute_reward(voltages, currents, count, tau)

            # Сохраняем значения value для GAE
            with torch.no_grad():
                val = agent.critic(torch.tensor(global_obs, dtype=torch.float32, device=device)).item()

            buffer['obs'].append(local_obs.copy())
            buffer['global_obs'].append(global_obs.copy())
            buffer['actions'].append(actions_raw)
            buffer['logprobs'].append(logprobs_raw)
            buffer['rewards'].append(reward)
            buffer['dones'].append(done)
            buffer['values'].append(val)

            local_obs, global_obs = next_local, next_global

            # Обновляем нормализатор
            state_norm.update(local_obs)

        # Последнее значение для GAE
        with torch.no_grad():
            last_val = agent.critic(torch.tensor(global_obs, dtype=torch.float32, device=device)).item()

        # Вычисляем GAE и returns
        advantages, returns = agent.compute_gae(
            buffer['rewards'], buffer['values'], buffer['dones'], last_val
        )

        # Подготавливаем данные
        train_data = {
            'obs': np.array(buffer['obs']),
            'global_obs': np.array(buffer['global_obs']),
            'actions': np.array(buffer['actions']),
            'logprobs': np.array(buffer['logprobs']),
            'returns': returns,
            'advantages': advantages
        }

        # Обновляем агента
        agent.update(train_data, batch_size=batch_size)

        if ep % 20 == 0:
            avg_r = np.mean(buffer['rewards'])
            print(f"Episode {ep}, Avg Reward: {avg_r:.3f}")

    print("Training finished.")